<a href="https://colab.research.google.com/github/jufabeck2202/KI-Lab/blob/main/Assignment_3_2_OWCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>